In [59]:
import pandas as pd
dicio = catalog.load('pred_geral_lista')
 

[01/28/24 02:01:47] INFO     Loading data from 'pred_geral_lista' (PartitionedDataset)...       ]8;id=445861;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=159111;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py#502\502]8;;\

In [60]:
lista = []

for key, value in dicio.items():
    df = value()
    df['modelo'] = key
    lista.append(df)


dfzao = pd.concat(lista)
dfzao

,index,sindex,tema,conjunto,SOMA_PREDS,PRED_GERAL,TARGET,modelo
0,0,14,2,1,735,839,840.0,TF_IDF_32
1,1,104,39,1,772,839,840.0,TF_IDF_32
2,2,80,13,1,703,520,520.0,TF_IDF_32
3,3,77,2,1,615,919,920.0,TF_IDF_32
4,4,40,54,1,763,640,640.0,TF_IDF_32
...,...,...,...,...,...,...,...,...
1521,1521,77,46,1,724,919,920.0,TF_IDF_64
1522,1522,16,43,1,707,680,680.0,TF_IDF_64
1523,1523,54,45,1,642,759,760.0,TF_IDF_64
1524,1524,125,81,1,677,759,760.0,TF_IDF_64


In [65]:
dicio['TF_IDF_32']()

,index,sindex,tema,conjunto,SOMA_PREDS,PRED_GERAL,TARGET
0,0,14,2,1,735,839,840.0
1,1,104,39,1,772,839,840.0
2,2,80,13,1,703,520,520.0
3,3,77,2,1,615,919,920.0
4,4,40,54,1,763,640,640.0
...,...,...,...,...,...,...,...
1521,1521,77,46,1,702,919,920.0
1522,1522,16,43,1,717,680,680.0
1523,1523,54,45,1,731,759,760.0
1524,1524,125,81,1,705,759,760.0


In [66]:
dicio['TF_IDF_64']()

,index,sindex,tema,conjunto,SOMA_PREDS,PRED_GERAL,TARGET
0,0,14,2,1,671,839,840.0
1,1,104,39,1,747,839,840.0
2,2,80,13,1,713,520,520.0
3,3,77,2,1,721,919,920.0
4,4,40,54,1,770,640,640.0
...,...,...,...,...,...,...,...
1521,1521,77,46,1,724,919,920.0
1522,1522,16,43,1,707,680,680.0
1523,1523,54,45,1,642,759,760.0
1524,1524,125,81,1,677,759,760.0


In [61]:
cohen_kappa_score(elef['SOMA_PREDS'],elef['TARGET'])

-1.592040828524155e-05

In [62]:
df1=
cohen_kappa_score(elef['PRED_GERAL'],elef['TARGET'])

0.3934496446500084

In [74]:
a = dicio['TF_IDF_32']()
b = dicio['TF_IDF_64']()

In [75]:
cohen_kappa_score(a['PRED_GERAL'],a['TARGET'])

0.3934496446500084

In [77]:
cohen_kappa_score(b['PRED_GERAL'],b['TARGET'])

0.3934496446500084